In [ ]:
import tensorflow as tf
from tensorflow import keras
from matplotlib import pyplot as plt
import pandas as pd

In [ ]:
keras.datasets.fashion_mnist.load_data()

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))

In [ ]:
def process_img(x, y):
    x = x /255
    return (x, y)

In [ ]:
dataset = dataset.map(process_img)

In [ ]:
dataset = dataset.batch(32)

In [ ]:
for i in dataset.take(1):
    img = i[0][0]
    plt.imshow(img)
    break

In [ ]:
dataset = dataset.shuffle(1000)

# build the CCN (Neural Network)

In [22]:
model = keras.models.Sequential([
        keras.Input(shape=[28, 28, 1]),

        # CNN block 1
        keras.layers.Conv2D(filters=64, kernel_size=7, padding="SAME", activation='relu'),
        keras.layers.MaxPooling2D(pool_size=2),

        # CNN block 2
        keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),

        # CNN block 3
        keras.layers.Conv2D(filters=128, kernel_size=3, activation='relu', padding="SAME"),
        keras.layers.MaxPooling2D(pool_size=2),

        # CNN block 4
        keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="SAME"),

        # CNN block 5
        keras.layers.Conv2D(filters=256, kernel_size=3, activation='relu', padding="SAME"),
        keras.layers.MaxPooling2D(pool_size=2),

        # Dense (Fully Connceted Layers)
        keras.layers.Flatten(),
        keras.layers.Dense(units=128, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=64, activation='relu'),
        keras.layers.Dropout(0.5),
        keras.layers.Dense(units=10, activation='softmax'),
])

In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 64)        3200      
                                                                 
 max_pooling2d (MaxPooling2  (None, 14, 14, 64)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 14, 14, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 7, 7, 128)         0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 7, 7, 256)         2

In [24]:
keras.utils.plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


# Compile the model 

In [25]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])

# Train the model 

In [ ]:
history = model.fit(dataset, 
                    epochs=10, 
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 633s 366ms/step - loss: 0.6813 - accuracy: 0.7616 - val_loss: 18.6712 - val_accuracy: 0.8712
Epoch 2/10
1033/1719 [=================>............] - ETA: 4:17 - loss: 0.4207 - accuracy: 0.8588

# plot learning curve 

In [ ]:
pd.DataFrame(history.history).plot(figsize=(8, 5))
plt.grid(True)
plt.show()